In [1]:
import random
from math import sqrt
from tqdm import tqdm
import time
from random import sample

def generate_random_permutation(n):
    permutation=[i for i in range(1,n+1)]
    pos1= random.randint(1,n-1)
    pos2= random.randint(1,n-1)
    permutation[pos1],permutation[pos2]=permutation[pos2],permutation[pos1]
    
    return permutation
        

In [2]:
class Chromosome:
    def __init__(self, paramProblem=None):
        self.__paramProblem=paramProblem
        self.__fitness=0
        self.__repres=generate_random_permutation(paramProblem['noCities'])
        self.__value_road=0
        
    
    @property
    def repres(self):
        return self.__repres
    
    @property 
    def fitness(self):
        return self.__fitness
    
    @property
    def value_road(self):
        return self.__value_road
    
    @repres.setter
    def repres(self, l=[]):
        self.__repres=l
    
    @fitness.setter
    def fitness(self, fit):
        self.__fitness=fit
        
        
    @value_road.setter
    def value_road(self, val):
        self.__value_road=val
    
    
    def crossover(self, c):
        
        pos1=random.randint(-1,self.__paramProblem['noCities'])
        pos2=random.randint(-1,self.__paramProblem['noCities'])
        
        if pos1>pos2:
            pos1,pos2=pos2,pos1
        
        newrepres=self.__repres[pos1:pos2]
        
        k=0
        for el in c.repres[pos2:]+c.repres[:pos2]:
            if (el not in newrepres):
                if len(newrepres)< (self.__paramProblem['noCities']-pos1):
                    newrepres.append(el)
                else:
                    newrepres.insert(k,el)
                    k+=1
            
        offspring= Chromosome(self.__paramProblem)
        offspring.repres=newrepres
        
        return offspring
    
    
    def mutation(self):
        pos1=random.randint(0,self.__paramProblem['noCities']-1)
        pos2=random.randint(0,self.__paramProblem['noCities']-1)
        
        if pos1>pos2:
            pos1,pos2=pos2,pos1
        
        el= self.__repres[pos2]
        
        del self.__repres[pos2]
        
        self.__repres.insert(pos1+1,el)
        
        
    def __str__(self):
        return "\nChromo: " + str(self.__repres) + " has fit: " + str(self.__fitness) + " with value: " + str(self.__value_road)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, c):
        return self.__repres == c.__repres and self.__fitness == c.__fitness
    
    def __ge__(self, c):
        return self.__fitness>= c.fitness

        

In [3]:


class Service:
    def __init__(self,filename):
        self.__filename=filename
        self.__paramProblem={}
        self.__Chromosomes=[]
        
    def read_instruction_txt(self):
        f = open(self.__filename, 'r')
        self.__paramProblem['noCities'] = int(f.readline())
        final_distances = []
        m_distances = []
        sum=0
        for i in range(1, self.__paramProblem['noCities'] + 1):
            distances = f.readline().split(',')
            for j in range(1, self.__paramProblem['noCities'] + 1):
                m_distances.append(int(distances[j - 1]))
                if i>j:
                    sum+=int(distances[j - 1])
            final_distances.append(m_distances)
            m_distances = []
        self.__paramProblem['matrix'] = final_distances
        self.__paramProblem['sum']= sum
    
    
    def read_instruction_tsp(self):
        f=open(self.__filename,'r')
        for i in range(0,6):
            f.readline()
        cities={}
        no_cities=0
        while True:
            line= f.readline()
            if line=='EOF':
                break
            info=line.split(" ")
            cities[int(info[0])]=(float(info[1]),float(info[2]))
            if int(info[0])>no_cities:
                no_cities=int(info[0])
                
        
        matrix=[[0 for x in range(0,no_cities)] for y in range(0,no_cities)]
        suma=0
        
        for i in range(0,no_cities-1):
            for j in range(i+1,no_cities):
                current_distance= sqrt((cities[i+1][0]-cities[j+1][0])**2 + (cities[i+1][1]-cities[j+1][1])**2)
                matrix[i][j]=matrix[j][i]= current_distance
                suma+=current_distance
        
        self.__paramProblem['noCities']= no_cities
        self.__paramProblem['matrix']= matrix
        self.__paramProblem['sum']=suma
        
    
    
    def generate_chromosomes(self,n):
        
        for i in range(n):
            chrom= Chromosome(self.__paramProblem)
            self.__Chromosomes.append(chrom)
            
    
    def get_chromosomes(self):
        return self.__Chromosomes
    
    
    
    
    def evaluate_chromosomes(self,chromosomes):
        for chrom in chromosomes:
            fit=0
            for j in range(0,self.__paramProblem['noCities']-1):
                fit+=self.__paramProblem['matrix'][chrom.repres[j]-1][chrom.repres[j+1]-1]
            fit+=self.__paramProblem['matrix'][chrom.repres[0]-1][chrom.repres[self.__paramProblem['noCities']-1]-1]
            
            chrom.fitness=round(((self.__paramProblem['sum']-fit)/self.__paramProblem['sum'])*10,3)
            chrom.value_road=fit
            
            
    def selection_n_reproduction(self,chromosomes):
        for i in range(0,len(chromosomes)):
            for j in range(i+1,len(chromosomes)):
                if chromosomes[i].fitness<chromosomes[j].fitness:
                    aux=chromosomes[i]
                    chromosomes[i]=chromosomes[j]
                    chromosomes[j]=aux
                    
            return chromosomes[0],chromosomes[1],chromosomes[3]
    
    
    def best_chromosome(self,choromosomes):
        
        for i in range(0,len(chromosomes)):
            for j in range(i+1,len(chromosomes)):
                if chromosomes[i].fitness<chromosomes[j].fitness:
                    aux=chromosomes[i]
                    chromosomes[i]=chromosomes[j]
                    chromosomes[j]=aux
        
        return chromosomes[0]
    

In [6]:
s=Service('hard_tsp_2.txt')

s.read_instruction_txt()


# s=Service('hardE.txt')

# s.read_instruction_tsp()

s.generate_chromosomes(5)
chromosomes=s.get_chromosomes()
s.evaluate_chromosomes(chromosomes)

for i in tqdm(range(3000)):
    new_chromosomes=[]
    
    [parent1,parent2,parent3]=s.selection_n_reproduction(chromosomes)
    
    offspring_1=parent1.crossover(parent2)
    offspring_2=parent2.crossover(parent1)
    offspring_3=parent3.crossover(parent1)
    offspring_7=parent2.crossover(parent3)
    
    offspring_5=parent1.crossover(parent3)
    offspring_4=parent2
    offspring_5.mutation()
    offspring_6=parent3.crossover(parent2)
    offspring_5.mutation()
    offspring_6.mutation()
    offspring_4.mutation()
    
    
    new_chromosomes.append(parent1)
    new_chromosomes.append(parent2)
    new_chromosomes.append(parent3)
    new_chromosomes.append(offspring_1)
    new_chromosomes.append(offspring_2)
    new_chromosomes.append(offspring_3)
    new_chromosomes.append(offspring_4)
    new_chromosomes.append(offspring_5)
    new_chromosomes.append(offspring_6)
    new_chromosomes.append(offspring_7)
        
    time.sleep(0.01)
    s.evaluate_chromosomes(new_chromosomes)
    chromosomes=new_chromosomes
    
    


print(s.best_chromosome(chromosomes))

100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:57<00:00, 52.48it/s]


Chromo: [9, 10, 11, 12, 13, 15, 14, 16, 19, 20, 18, 17, 21, 22, 26, 23, 24, 25, 1, 2, 3, 4, 6, 5, 7, 8] has fit: 9.716 with value: 957
